<a href="https://colab.research.google.com/github/asdfasdf001234/2024-1-MLPRJ/blob/main/DNN%EA%B5%AC%EC%A1%B0_2%EB%B6%84%EB%A5%98_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import seaborn as sns

# pretrained 관련
import torch
import torchvision.transforms as v2
from torchvision import models

import torch
import numpy as np
import random

def set_random_seed(seed_value):
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)

# Set a random seed value
seed_value = 42
set_random_seed(seed_value)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torch.utils.data import Dataset , DataLoader
from torchvision import datasets
from torchvision.transforms import v2
from PIL import Image
from glob import glob
data_dir = "/content/drive/MyDrive/Colab Notebooks/MLPRJ/Data"

In [ ]:
import pandas as pd
import os
from glob import glob

def create_dataframe(data_path, label_list, data_type):
    df = pd.DataFrame({"path": [], "label": [], "class_id": []})
    img_list = glob(os.path.join(data_path, '*.jpg'))

    for img in img_list:
      file_name = os.path.splitext(os.path.basename(img))[0]
      label_index = int(file_name[0]) - 1
      if label_index == 0 or label_index == 2:   #웜톤
        label_index = 0                          #0으로 통일
      else:                                      #쿨톤
        label_index = 1                          #1으로 통일
      if 0 <= label_index < len(label_list):
        label = label_list[label_index]
        new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})
        df = pd.concat([df, new_data], ignore_index=True)


    df[["path"]] = df[["path"]].astype(str)
    df[["label"]] = df[["label"]].astype(str)
    df[["class_id"]] = df[["class_id"]].astype(int)

    return df


In [ ]:
# 기존 example 코드
train_path = data_dir + '/train'
valid_path = data_dir + '/val'
test_path = data_dir + '/test'
label_list = ['warm', 'cool']

train_df = create_dataframe(train_path, label_list, 'training')
val_df = create_dataframe(valid_path, label_list, 'validation')
test_df = create_dataframe(test_path, label_list, 'testing')

In [ ]:
print(f"train_data: {len(train_df)}")
print(f"val_data:{len(val_df)}")
print(f"test_data:{len(test_df)}")

train_data: 413
val_data:137
test_data:137


In [ ]:
class BaseDataset(torch.utils.data.Dataset):
    def __init__(self , dataframe , transforms_):
        self.df = dataframe
        self.transforms_ = transforms_

    def __len__(self):
        return len(self.df)

    def __getitem__(self ,index):
        img_path = self.df.iloc[index]['path']
        img = Image.open(img_path).convert("RGB")
        transformed_img = self.transforms_(img)
        class_id = self.df.iloc[index]['class_id']
        return transformed_img , class_id

In [ ]:
Transforms = v2.Compose([
    v2.Resize((224,224)),
    v2.RandomRotation(degrees=10),
    v2.RandomHorizontalFlip(p=0.8),
    v2.ScaleJitter(target_size=(224,224)),
    v2.RandomAffine(degrees=45),
    #v2.RandomErasing(),
    v2.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.)),
    v2.ElasticTransform(alpha=250.0),
    #v2.ColorJitter(0.5, 0.5),
    #v2.RandomResizedCrop(size=(224, 224), antialias=True),

    v2.Resize((64,64)), #사이즈를 64*64
    v2.PILToTensor(),
    v2.ToDtype(torch.float32),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [ ]:
BATCH_SIZE = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#get dataloader

train_dataset = BaseDataset(train_df, Transforms) # train_transforms
val_dataset = BaseDataset(val_df, Transforms)
test_dataset = BaseDataset(test_df, Transforms)

train_loader = DataLoader(train_dataset , batch_size=BATCH_SIZE , shuffle = True)
val_loader = DataLoader(val_dataset , batch_size=BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_dataset , batch_size=BATCH_SIZE, shuffle = True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        # 64*64에 맞춰 레이어 수정
        self.fc1 = nn.Linear(64*64*3, 10000)
        self.fc2 = nn.Linear(10000, 50000)
        self.fc3 = nn.Linear(50000, 10000)
        self.fc4 = nn.Linear(10000, 5000)
        self.fc5 = nn.Linear(5000, 1024)
        self.fc6 = nn.Linear(1024, 512)
        self.fc7 = nn.Linear(512, 64)
        self.fc8 = nn.Linear(64, 32)
        self.fc9 = nn.Linear(32, 2)

        self.b1 = nn.BatchNorm1d(10000)
        self.b2 = nn.BatchNorm1d(50000)
        self.b3 = nn.BatchNorm1d(1024)
        self.b4 = nn.BatchNorm1d(512)
        self.b5 = nn.BatchNorm1d(64)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.to(device)
        x = self.flatten(x)
        x = F.relu(self.b1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.b2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.b1(self.fc3(x)))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        x = F.relu(self.b3(self.fc5(x)))
        x = self.dropout(x)
        x = F.relu(self.b4(self.fc6(x)))
        x = self.dropout(x)
        x = F.relu(self.b5(self.fc7(x)))
        x = F.leaky_relu(self.fc8(x))
        x = self.fc9(x)
        return x


In [ ]:
# hyperparameter 설정
import torch.optim as optim
FcModel = NeuralNetwork()
criterion = nn.CrossEntropyLoss() # loss function
optimizer = optim.Adam(FcModel.parameters(), lr=0.5 )

lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

EPOCHS = 20 # the number of epochs
n_batch = 32 # the number of batches


In [ ]:
def train(dataloader , model , loss_fn , optimizer , lr_scheduler=None):
    size = 0
    num_batches = len(dataloader)

    model.train()
    epoch_loss , epoch_correct = 0 , 0

    for i ,(data_ , target_) in enumerate(dataloader):
        #===================================================#
        #모델 예측값과 실제 값
        data_, target_ = data_.to(device), target_.to(device)
        size += data_.size(0)

        pred = model(data_)
        loss = loss_fn(pred, target_)
        epoch_loss += loss.item()
        epoch_correct += ( pred.argmax(1) == target_ ).type(torch.float).sum().item()

        #역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        #===================================================#

    if lr_scheduler != None:
        lr_scheduler.step()

    return epoch_correct/size , epoch_loss / num_batches

In [ ]:
def test(dataloader , model , loss_fn):
    size = 0
    num_baches = len(dataloader)
    epoch_loss , epoch_correct= 0 ,0

    with torch.no_grad(): # grad 연산 X
        model.eval() # evaluation dropout 연산시
        for i, (data_ , target_) in enumerate(dataloader):

            #========================================#
            data_, target_ = data_.to(device), target_.to(device)
            size += data_.size(0)
            pred = model(data_)
            loss = criterion(pred, target_)
            epoch_loss += loss.item()
            epoch_correct += ( pred.argmax(1) == target_ ).type(torch.float).sum().item()

            #========================================#

    return epoch_correct/size  , epoch_loss / num_baches

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
best_loss = 0

In [ ]:
FcModel.cuda()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=12288, out_features=10000, bias=True)
  (fc2): Linear(in_features=10000, out_features=50000, bias=True)
  (fc3): Linear(in_features=50000, out_features=10000, bias=True)
  (fc4): Linear(in_features=10000, out_features=5000, bias=True)
  (fc5): Linear(in_features=5000, out_features=1024, bias=True)
  (fc6): Linear(in_features=1024, out_features=512, bias=True)
  (fc7): Linear(in_features=512, out_features=64, bias=True)
  (fc8): Linear(in_features=64, out_features=32, bias=True)
  (fc9): Linear(in_features=32, out_features=2, bias=True)
  (b1): BatchNorm1d(10000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b2): BatchNorm1d(50000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b5): 

In [ ]:
for epoch in tqdm(range(EPOCHS)):
    train_acc , train_loss = train(train_loader ,
                                   FcModel ,
                                   criterion ,
                                   optimizer,
                                   lr_scheduler )

    val_acc , val_loss = test(val_loader , FcModel , criterion)
    print(f'epoch:{epoch} \
    train_loss = {train_loss:.4f} , train_acc:{train_acc:.4f} \
    val_loss = {val_loss:.4f} , val_acc:{val_acc:.4f} \
    learning rate: {optimizer.param_groups[0]["lr"]}')


    if val_loss < best_loss:
        counter = 0
        best_loss = val_loss
        torch.save(FcModel.state_dict() , "checkpoints/NN_best.pth")


  5%|▌         | 1/20 [00:18<05:58, 18.89s/it]

epoch:0     train_loss = 8.0464 , train_acc:0.5424     val_loss = 1538.3453 , val_acc:0.3650     learning rate: 0.45


 10%|█         | 2/20 [00:37<05:40, 18.90s/it]

epoch:1     train_loss = 0.7903 , train_acc:0.6392     val_loss = 14.8538 , val_acc:0.3650     learning rate: 0.405


 15%|█▌        | 3/20 [00:56<05:22, 18.97s/it]

...

epoch:17     train_loss = 0.6544 , train_acc:0.6368     val_loss = 0.6680 , val_acc:0.6350     learning rate: 0.0750473176484996


 95%|█████████▌| 19/20 [05:58<00:18, 18.89s/it]

epoch:18     train_loss = 0.6717 , train_acc:0.6271     val_loss = 0.7413 , val_acc:0.6350     learning rate: 0.06754258588364964


100%|██████████| 20/20 [06:17<00:00, 18.88s/it]

epoch:19     train_loss = 0.6701 , train_acc:0.6368     val_loss = 0.6886 , val_acc:0.6350     learning rate: 0.06078832729528468


In [ ]:
test_acc , val_loss = test(test_loader , FcModel , criterion)
print(test_acc)

0.635036496350365
